In [149]:
import pandas as pd
import itertools
import numpy as np

In [150]:
# Data Preprocessing
path = "../../"
path2 = "..\../Correlation_Analysis\WQ0416/"
#path_gis = '/Users/cong/Downloads/WaterQualityProject/SEACAR_WQ_Pilot-selected/GIS_data/'
path_gis = '..\../GIS_data/'
#WQ_file = path_gis + 'OEATUSF_Geospatial_TempSeasons.csv'
WQ_file = path2 + '..\../OEATUSF_Geospatial_TempSeasons_update_0510.csv'
#WQ_file_update = path_gis + 'OEATUSF_Geospatial_TempSeasons_update.csv'
WQ_file_update = path2+ 'OEATUSF_Geospatial_TempSeasons_update_0510_output.csv'

dfDis_orig = pd.read_csv(path + r"discrete_0214.csv")
dfCon_orig = pd.read_csv(path + r"continuous_0214.csv")

C:\Users\yi\AppData\Local\Temp/ipykernel_47220/1027411217.py:11: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  dfDis_orig = pd.read_csv(path + r"discrete_0214.csv")


Add new columns to the OEATUSF_Geospatial...

In [151]:
count_table = pd.read_csv(WQ_file)

count_table.head()

,Unnamed: 0.1,Unnamed: 0,param,ma,st_Year,season,med_seamo_ma,s_start,s_end,date_range,covariates,Processed,Total Nitrogen,Dissolved Oxygen,Salinity,Secchi Depth,Turbidity
0,0,147,temp,Estero Bay Aquatic Preserve,2022,Summer,7-Jun,6/7/2022,8/26/2022,80,bathymetry+LDI+popden,1,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,NaN,NaN,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...
1,1,71,temp,Biscayne Bay Aquatic Preserve,2022,Summer,1-Jun,6/1/2022,8/26/2022,86,bathymetry+LDI+popden,1,NaN,NaN,NaN,NaN,NaN
2,2,70,temp,Biscayne Bay Aquatic Preserve,2022,Spring,2-Mar,3/2/2022,5/31/2022,90,bathymetry+LDI+popden,1,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,NaN,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...
3,3,146,temp,Estero Bay Aquatic Preserve,2022,Spring,27-Feb,2/27/2022,6/6/2022,99,bathymetry+LDI+popden,1,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,NaN,NaN,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...
4,4,69,temp,Biscayne Bay Aquatic Preserve,2022,Winter,8-Feb,2/8/2022,3/1/2022,21,bathymetry+LDI+popden,1,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,NaN,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...


Combine discrete and continuous dataframe

In [152]:
merged_df = pd.concat([dfDis_orig, dfCon_orig])
merged_df.shape

(8933025, 16)

In [153]:
count_table['ma'] = count_table['ma'].replace('Guana River Marsh Aquatic Preserve','Guana Tolomato Matanzas National Estuarine Research Reserve')

Create two new columns to represent start and end date of each season

In [154]:
# count_table['s_start'] = count_table['med_seamo_ma'].str.replace(' ','-') +'-'+ count_table['st_Year'].astype(str)
# count_table['s_start'] = pd.to_datetime(count_table['s_start'])

# s_end_ls = []
# for area in count_table['ma'].unique():
#     table = count_table[count_table['ma']==area].reset_index()
#     table_len=table.shape[0]
#     for idx,row in table.iterrows():
#         if idx+1 < table_len:
#             row2 = table.loc[idx+1]
#             s_end = row2['s_start'] - pd.Timedelta(days=1)
#         elif idx+1 == table_len:
#             s_end = np.nan
#         s_end_ls.append(s_end)

# count_table['s_end']=s_end_ls

In [155]:
count_table['s_start'], count_table['s_end'] = pd.to_datetime(count_table['s_start']), pd.to_datetime(count_table['s_end'])

In [156]:
merged_df['date'] = pd.to_datetime(merged_df['timestamp']).dt.date

In [157]:
count_table['s_start'], count_table['s_end'] = count_table['s_start'].dt.date,count_table['s_end'].dt.date

In [158]:
ls_count = pd.DataFrame(columns=['Dissolved Oxygen','Salinity','Secchi Depth','Total Nitrogen','Turbidity'])
#ls_count = []
para_ls = merged_df['ParameterName'].unique
for idx,row in count_table.iterrows():
    #ma, s_start,s_end = row['ma'],row['st_Year'],row['s_start'], row['s_end']
    merged_df2 = merged_df.loc[(merged_df['ManagedAreaName'] == row['ma'])& 
                               (merged_df['Year'] == row['st_Year'])&
                               (merged_df['date'] > row['s_start'])&
                               (merged_df['date'] < row['s_end'])]
    merged_df3= merged_df2.groupby(['ParameterName','Latitude_DD','Longitude_DD']).size().reset_index().rename(columns={0:'Count'})
    merged_df4 = merged_df3.groupby(['ParameterName']).size().reset_index().rename(columns={0:'Count'})
    if merged_df3.shape[0]>0:
        merged_df4 = pd.DataFrame(dict(zip(list(merged_df4.ParameterName), list(merged_df4.Count))),index = [0])
        ls_count = pd.concat([ls_count, merged_df4], axis=0)   
    elif merged_df3.shape[0]==0:
        merged_df4 = pd.DataFrame({'Dissolved Oxygen': [np.nan],'Salinity': [np.nan],'Secchi Depth': [np.nan],'Total Nitrogen': [np.nan],'Turbidity': [np.nan]})
        ls_count = pd.concat([ls_count, merged_df4], axis=0)

In [160]:
ls_count.reset_index(drop=True, inplace=True)

In [161]:
count_table = pd.concat([count_table, ls_count], axis=1).drop(columns = ['Unnamed: 0.1'])

In [162]:
count_table.to_csv(WQ_file_update, index=False)